In [1]:
from IPython.core.display import HTML
import urllib2
HTML(urllib2.urlopen('http://bit.ly/1Bf5Hft').read())

## Fraud identification from Enron Dataset

Formed in 1985 from a merger of Houston Natural Gas and Internorth, Enron Corp.
was the first nationwide natural gas pipeline network and over time became  one of the biggest companies in the United States with billions of dollars in reveneues. When the recession hit in 2000, Enron had significant exposure to the most volatile parts of the market and sustained heavy losses. Rather than disclose its true condition to public investors, as the law requires, Enron falsified its accounts. To hide the financial losses of the trading business and other operations of the company, the management at Enron used a technique called *mark-to-market* accounting which enabled Enron to write off unnprofitable activities without hurting its bottom line.
<br>
<br>
When this corporate fraud was exposed, Enron suddenly collapsed and went bankrupt.  A significant amount of Enron data (emails and financial data) were entered into public record as a result of Federal investigation. The goal of this project is to build a classifier that can predict Enron employees involved in the fraud based on the public Enron financial and email dataset. A "Person of Interest" (POI) is someone who was indicted or settled without admitting guilt (paid a fine but no criminal charges) or someone who testified in exchange for immunity. The email and finance data is combined into a single dataset, which will be used to build a POI classifier. 
<br>
<br>
One potential future application is to build a similar person of interest classifier for fraud detection using the techniques explored in this project. (Granted that fraud comes in different types and sizes, the idea here is to build techniques and ask the right questions that may help us with fraud detection in the future.)
<br>
<br>
#### Data Overview, Cleaning and Preparation

##### Overview
The data is available as a nested dictionary with keys being the list of people and values being a dictionary with values for different features. The dataset consisted of 146 records with 19 features (14 financial features, 5 e-mail features) and 1 label POI. Overall, there were 18 POIs identified beforehand and labelled accordingly. One observation is that there are numerous NaNs in both the email and financial fields. According to the official pdf documentation for the financial data, values of NaN represent 0 and not unknown quantities. However, for the email data, NaNs are unknown information. The full list of financial features comprieses of ```bonus```, ```deferral_payments```, ```deferred_income```, ```director_fees```, ```exercised_stock_options```, ```expenses```, ```loan_advances```, ```long_term_incentive```, ```other```, ```restricted_stock```, ```restricted_stock_deferred```, ```salary```, ```total_payments``` and  ```total_stock_value```. The full list of e-mail features comprises of ```from_messages```, ```from_poi_to_this_person```, ```from_this_person_to_poi```, ```shared_receipt_with_poi```, ```and to_messages```. The feature labels are fairly self explanatory and all of them are numerical features. 
<br>
<br>
##### Cleaning
The first thing I did was to check if all the entries make sense. Printing out the dictionary keys revealed two records which did not seem right; ``` TOTAL``` and ```THE TRAVEL AGENCY IN THE PARK```. The records in dataset are for individuals and these two names stand out. The entry under ```TOTAL``` comprises of the total of other records while ```THE TRAVEL AGENCY IN THE PARK``` seems like an organization and not a person. (In fact, that happened to be the name of a travel agency owned by the younger sister of Mr. Kenneth Lay, the chairman of Enron at the time of its downfall. ) As these two entries either don't give new information or are not directly related to the problem at hand, let us remove them. 
<br>
<br>
As a second step, let us remove any entries that has all features as NaN as this does not give us anything useful for predictve modeling. Note that each entry has 20 features (One feature not included in the list displayed above is ```email-id```. Knowing e-mail id is useless for predicting a person of interest and later, we will remove this feature altogether) and 1 label i.e the 'feature' list for the raw data has length 21. (The raw data has ```poi``` as the first 'feature'. In fact, this is the label we are wanting to predict.) To find a data entry with all *actual* features as NaN's, we check if the number of NaNs is 20. (Indicating that all features are NaN.) We remove the corresponding entry. It turns out that the data entry for ```LOCKHART EUGENE E``` has all features as NaN and hence this person is removed from the data set. 
<br>
<br>
As a next step, I spot checked with financial data PDF file (`enron61702insiderpay.pdf`) and observed that only two features are allowed to take negative values, ```deferred_income``` and ```restricted_stock_deferred```. Any data entry that has negative values for features not in the this list merits a closer look. Using this reasoning, the entries for ```BELFER ROBERT``` and ```BHATNAGAR SANJAY``` are flagged. It turns out that the data entered in to ```final_project_dataset.pkl```  (which is the raw data in a dictonary format) was shifted one column to the right relative to the PDF. I reconstructed both Sanjay's and Robert's data based on the PDF. 
<br>
<br>
There are no more errors that are apparent from examination of the dataset and we move on to the next phase. Note that some of the features are linear combinations of others (```total_payments```, ```total_stock_values```) or aggregate of others and it might be tempting to remove them. But since our dataset is small, we do not want to lose information and for this reason I choose to keep these features in the dataset. When feeding our data in to a machine learning algorithm, we can perform feature selection or extraction to select only those features in our data that are most useful. For now, we keep the features. 
<br>
<br>
##### Preparation

To improve the predictive power of our analysis, I choose to engineer two new features ```bonus_to_salary``` and ```bonus_to_total``` which repectively are the ratio of bonus to salary and bonus to total. This seems intuitive to me since a large bonus to salary/ total ratio might indicate that the person is on the fraud and is perhaps being rewarded for keeping it a secret. (It might also happen that a person has performed spectacularly well and is being rewarded for her/ his performance with a generous bonus, but it is more likely that the bonus is not awarded for this reason.) In any case, engineering these two new features seem logical to me. 
<br>
<br>
As discussed earlier, I drop the ```e-mail_id``` feature as that is not useful for analysis. 
<br>
<br>
We now move on to the next phase i.e choosing what features to feed in our classification algorithms. 
<br>
<br>
#### Initial Feature Selection and Performance metrics
##### Initial Feature Selection
I now make a modeling choice and altogether drop the features ```from_poi_to_this_person```, ```from_this_person_to_poi``` and ```shared_receipt_with_poi``` from the dataset. ***The way these features have been engineered in the data set to be  then fed to ML algorithm is incorrect: by doing so we are always peeking into the test set (even worse, at the test set labels) and that is obviously not correct. That happens because to calculate, for instance, the number of emails sent to a POI, we are using all the POIs in a dataset and not saving the test ones from the count. For a test dataset, we cannot obviously use such feature, since we do not know who in the test set is a POI. The training set should not incorporate any information from the test set.***
<br>
<br>
I remove these features from the dataset as otherwise we would be peeking into the testset labels and this is wrong. 
<br>
<br>
The final dataset we will use for training and evaluation has 143 records with each record having 18 features and 1 label. Out of the 143 records (persons), 18 are persons of interest. The goal of our project is to create a classifier that can identify persons of interest. Hence a very important question we need to annswer before we proceed is to decide on a performance metric. 
<br>
<br>
##### Choice of performance metrics
*Accuracy* would be the first obvious choice for a performance metric (i.e how many entries are correctly identified.) This would not be a good choice because our dataset is heavily biased in favor of non persons of interest. For example, if a classifier were to guess that all of the samples in the cleaned dataset were not persons of interest, it would have an accuracy of 87.41%. However, this clearly would not satisfy the objective of this investigation which is to create a classifier that can identify *persons of interest*. Therefore, different metrics are needed to evaluate the classifiers to to gauge performance. The two selected for use in this project are Precision and Recall.
<br>
<br>
***Precision***  
Precision is the number of correct positive classifications divided by the total number of positive labels assigned. In other words, it is the fraction of persons of interest predicted by the algorithm that are actually persons of interest. Precision is defined mathematically as 
$$ precision = \frac{\text{ true positives} }{\text{true positives} + \text{ false positives}}$$

***Recall***

Recall is the number of correct positive classifications divided by the number of positive instances that should have been identified. In other words, it is the fraction of the total number of persons of interest in the data that the classifier identifies. Mathematically, recall is defined as
$$recall = \frac{\text{ true positives} }{\text{true positives} + \text{ false negatives}}$$
<br>
Precision is also known as positive predictive value while recall is called the sensitivity of the classifier. A combined measured of precision and recall is the F1 score. It is the harmonic mean of precision and recall. Mathematically, the F1 score is defined as:

$$\text{F1 Score} = 2 \frac{(\text{precision} \times \text{recall})}{\text{precision} + \text{recall}}$$
<br>
*For this project, the objective was a precision and a recall both greater than 0.3.*
<br>
<br>
#### Algorithms, tuning and validation 
We now select 4 different algorithms for classfication and tune the parameters so as to choose the *best* performing algorithm. Before we go on to proceed with the algorithm selection, a few remarks are in order. 
<br>
<br>
 If we have lots of data, we would split our data into disjoint parts, use training data for selection of algorithm and parameter settings, and a test set to assess how well the fit model generalizes to new data. Typically, that training data is split into further parts in order to derive an optimal parameter setting. One part is a training set that is used to fit models and the other part is a validation set to compute performance. By splitting data in this way, we can try many parameter combinations on the training set to find one that we believe will generalize best to novel data points. And by withholding a test set completely from the full training process (training and validation sets), we get an idea of this generalizability, since the test set data is truly novel for the fit model.
<br>
<br>
If we have plenty of data available, then we can consider this three disjoint set approach relatively easily. But if our available data is decreased, then we may not have enough information to satisfactorily divide the data in the simple matter above. We run a higher risk of having spurious associations selected by our model if we are unlucky with the way that the training set is selected. To get around this, instead of just one training set and one validation set, we perform multiple splits on the training data and have multiple training sets and multiple validation sets to decide on our final model parameters. 
<br>
<br>
The idea behind this *cross-validation* is that the parameter set that has the best overall performance across all of the training-validation splits should have the best chance of generalizability, since that parameter set has demonstrated a capability of maintaining good performance even if the training data changes. In *sklearn*, the ```GridSearchCV``` class is a convenient way of pulling off this form of parameter selection. When fed training data, it will perform multiple training-validation splits and fit a selected algorithm or pipeline to these splits, selecting a parameter set that has the best overall performance on the validation sets. 
<br>
<br>
By default, ```GridSearchCV``` performs 3-fold cross-validation, but we can also set different cross-validation or scoring functions. Since F1-score incorporates both recall and precision, we will use this scoring function for the validation step.  To counter the small size of the dataset, and its imbalance, we can use another method of validating the dataset called ```StratifiedShuffleSplit``` (it is a way of creating many test/train splits in the data, so that we train and test our model on multiple partitions of the data).
<br>
<br>
When we use ```GridSearchCV```, the validation is performed internally. We don't use a test/train split to validate our model. The optimal model is selected by ```GridSearchCV``` based on the internal train/validation subsets that it creates automatically. If we fit our ```GridSearchCV``` instance on a training subset, all that we will be doing is passing it less data to create its internal subsets with - which means that, with a small dataset like the one that we are using, this strategy makes it less likely that we will select an optimal model. Hence we fit ```GridSearchCV``` instance on the entire dataset. 
<br>
<br>
##### Algorithms
We will create a ```GridSearchCV``` instance for each model we are tuning. We will do model selection and parameter tuning in a single step using ```StratifiedSplit``` and ```GridSearchCV```. We will also standardize the data before feeding it to an algorithm. Note that not all classifiers require scaling, but we will do it anyway since it is good to  standardize
all the features. 
<br>
<br>
```
## Create a StratifiedShuffeSplit of the dataset which will we passed as an argument 
to GridSearchCV through a pipeline object.
sss = StratifiedShuffleSplit(n_splits=100, test_size=0.3, random_state=0) # 100 splits with 70-30 split

## Scaling creates non-dimensional features so that features with larger units 
## do not have an undue influence on the classifier as would be the case if 
## the classifier uses some sort of distance measurement (such as Euclidean 
## distance) as a similarity metric.
scaler = StandardScaler()
features = scaler.fit_transform(features)
```

We now use the following algorithms. 
<br>
<br>
1) **Gaussian Naive Bayes**
<br>
<br>
Let's start off with a simple algorithm *Gaussian Naive Bayes* for classification. Since Gaussian Naive Bayes assumes independence of features, it is important to do feature selection before feeding the data in to a Naive Bayes classifier. We can use PCA to reduce the dimensionality (and select linearly independent features.) We will use *sklearn's* ```pipeline``` obejct to perform sequence of different transformations of dataset before applying final estimator. The purpose of the pipeline is to assemble several steps that can be cross-validated together while setting different parameters.
<br>
<br>
The followling snippet of code enables us to select the best number of features for PCA based on Gaussian Naive Bayes performance  as validated by ```StratifiedShuffleSplit``` using F1 metric for scoring.

```
## Gaussian Naive Bayes
pipeline_gnb = Pipeline([("scale", StandardScaler()),
                   ('pca', PCA()),
                   ('classify', GaussianNB())]
param_grid_gnb = [{'pca__n_components': list(range(1, 19))}]
gs_gnb = GridSearchCV(pipeline_gnb, param_grid=param_grid_gnb, scoring='f1', cv = sss)
gs_gnb.fit(features, labels)
```

2) **Linear Discriminant Analysis**
<br>
<br>

We now consider Linear Discriminant Analysis (LDA). Unlike Naive Bayes, we do not require independence of features but we do assume that the conditional distribution of features given labels follows a Gaussian distribution with the same covariance matrix for different class labels. In a highly imbalanced class labels scenario, it is unlikely that this assumption will be satisfied. Hence, LDA is likely not to work very well. We optimize over the number of features to get the best performance out of LDA. 

```
pipeline_lda = Pipeline([('pca', PCA()),
                         ('classify', LinearDiscriminantAnalysis())])
param_grid_lda = [{'pca__n_components':list(range(1, 19))}]
gs_lda = GridSearchCV(pipeline_lda, param_grid=param_grid_lda, scoring='f1', 
                      cv=sss)
gs_lda.fit(features, labels)
```

3) **Logistic Regression** 
<br>
<br>

We now consider Logistic Regression with L1 penalty since L1 penalty promotes sparsity and is one way to achieve feature selection. We consider various regularization constants and tune over the grid of the regularization parameter. 
<br>
<br>
In *sklearn*, ```LogisticRegression``` comes with a built-in method of handling imbalanced classes. If we have highly imbalanced classes and have not addressed it during preprocessing, we have the option of using the ```class_weight``` parameter to weight the classes to make certain we have a balanced mix of each class. Specifically, the balanced argument will automatically weigh classes inversely proportional to their frequency. We also fix a random state in the Logistic Regression object to fix the random state used by the default optimization solver ```liblinear```.

```
## Logistic Regression with L1 penalty
pipeline_lgr = Pipeline([
                       ('classify', LogisticRegression(random_state=0, class_weight='balanced'))
                       ])
param_grid_lgr =  [{'classify__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]}]
gs_lgr = GridSearchCV(pipeline_lgr, param_grid=param_grid_lgr, scoring='f1', cv = sss)
gs_lgr.fit(features, labels)```


4) **Decision Tree**
<br>
<br>
Use ```GridSearchCV```, we will automate the process of finding the optimal number of features to be fed in to a Decision Tree algorithm. Decision-tree learners can create over-complex trees that do not generalise the data well. Mechanisms such as setting the maximum depth of the tree are necessary to avoid this problem. We perform feature selection using ```SelectKBest()``` which removes all but the ```k``` highest scoring features and is a form of univariate feature slection. We will also select the best hyper-parameter for the Decision Tree algorithm (```criterion```, ```min_samples_split``` and ```max_depth``` using ```GridSearchCV```. 
<br>
<br>
In the decision tree object, we will set ```class_weight``` to balanced to ensure we have a balanced mix of each class. 

```
pipeline_dt = Pipeline([
    ('select_features', SelectKBest()),
    ('classify', DecisionTreeClassifier(random_state=0, class_weight='balanced'))
])
param_grid_dt = [
    {
        'select_features__k': np.arange(1, 18), 
        'classify__criterion' : ['gini', 'entropy'],
        'classify__min_samples_split' : [2, 4, 6, 8, 10, 20],
        'classify__max_depth' : [None, 5, 10, 15, 20]
    }
]
gs_dt = GridSearchCV(pipeline_dt, param_grid=param_grid_dt, scoring='f1',
                     cv=sss)
gs_dt.fit(features, labels)
```

At this point, we can perhaps even try a Random Forest classifier to mitigate some of the problems of a Decision Tree classfier. But tuning it over different parameters (the same parameters as in the Decision Tree Classifier along with the  number of trees in the forest) is too computationally intensive and takes hours. 





#### Final Model 

To choose the best performing model out of the 4 algorithms we cross validated for, we record the F1 score of the best estimator for each of the 4 best classifiers for each algorithm. The F1 scores are as below. 

```
{'LDA': 0.30338378288378293, 
'Logistic Regression': 0.32303657166034339, 
'Gaussian Naive Bayes': 0.30097638910796803, 
'Decision Tree Classifer': 0.33244482461123331}
```
Based on this we select the Decision Tree classifier with various parameters tuned to achieve a high F1 score.

``` 
gs_dt.best_estimator_
Pipeline(steps=[('select_features', SelectKBest(k=15, score_func=<function f_classif at 0x000000000C6AE0B8>)), 
('classify', DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=20, min_weight_fraction_leaf=0.0,
            presort=False, random_state=0, splitter='best'))])
```
We observe that ```GridSearchCV``` selects top 15 features based on ```SelectKBest()``` algorithm. Let us display the features that end up in the final classfication model. 
```
 'total_stock_value',
 'exercised_stock_options',
 'bonus',
 'salary',
 'deferred_income',
 'bonus_to_salary',
 'long_term_incentive',
 'total_payments',
 'restricted_stock',
 'bonus_to_total',
 'loan_advances',
 'expenses',
 'other',
 'director_fees',
 'to_messages'
 ```

This is the classifier object that we pass on to the testing script *tester.py* which uses StratifiedShuffleSplit with a *different* initial random state and *different train/ test split*. This avoids the classic mistake of overfitting by using the same test/ train aplit for validation and testing. 

<br>
<br>
The output from *tester.py*
for the final evaluation of the classification Decision Tree model is shown below. 

```
Accuracy: 0.77087      
Precision: 0.31097      
Recall: 0.59100 
F1: 0.40752     
F2: 0.50081
Total predictions: 15000       
True positives: 1182    
False positives: 2619   
False negatives:  818   
True negatives: 10381
```

Our final tuned Decision Tree classification model achieves a precision of 0.31 and a recall of .591.  A precision score of 0.31 means that of the individuals labeled by my model as persons of interest, 31% of them were indeed persons of interest. A recall score of 0.591 means that my model identified 59.1% of persons of interest present in the entire dataset. Though precision and recall are the primary two metrics we use for evaluation, recall is a more important metric in a fraud detection problem like ours. It is okay to tag innocent people as guilty since that can be rectified by human follow up but it would be very bad if we our algorithm tags guilty people as innocent. Hence, recall is a far more important metric in classification problems of this nature. 

<br>
<br>

#### Conclusion 

The objective of this project was to build a Person of Interest Classifier based on financial and email data from Enron Fraud. Once this question was formulated,  the next step was data collection and cleaning. The biggest challenge I faced was the relative lack of data, so to speak. Lots of features had missing values and it was important not to drop all those features since we did not have much data. The dataset was messy and needed some cleaning. This project was an accurate representation of working with real world data which is often incomplete and messy. 
<br>
<br>
Once the data was fixed, there were some features that I dropped since they were either not useful for predictive modeling or were engineered in the original data set by peeking in to the test data. I then created a couple of features of my own which intuitively held the promise of enabling better predictions. 
<br>
<br>
Once the data was prepared, it was ready to be fed in to classification models. But first, we needed to decide on the performance metric. *Precision* and *recall* were used as metrics as those make more sense in a highly imbalanced dataset as ours. We also standardized the features (mean zero and unit variance) before feeding them in to the models. 4 different classification algorithms were tried out.  Parameter tuning and cross validation were done in a single step using a `Pipeline` object using ```GridSearchCV``` and ```StratifiedShuffleSplit``` and the best performing model was chosen for testing. 
<br>
<br>
The final classification model achieved a test precision of 0.31 and a test recall of 0.591 and was obtained by repeated splits of the data in to training and testing sets and considering the performance of the model on each such split. Due to the small size of the available data, even a minor change such as altering the random seed when doing a train-test split can have significant effects on the performance of the algorithm which must be accounted for by testing over many subsets and calculating the average performance.
<br>
<br>
This project showed me that human intuition about a situation can  match the results returned by a machine learning model. For example, I hypothesized that the ```total_stock_value```, `bonus` and `salary` or a combination thereof would be highly predictive indicators of fradulent activity and this was confirmed by the machine learning algorithm. I was also surprised at the relatively poor performance of parametric machine learning algorithms and could have done more in this project by choosing more powerful and non-parametric algorithms. This could have resulted in a more powerful classfier but overall, I was happy with the performance of the Decision Tree classifier.